In [8]:
using Random
using StatsPlots
using LightGraphs
using FreqTables
using Plots; gr() # Using the Plotly Backend

Plots.GRBackend()

In [9]:
# all possible states a person can be in
@enum Status susceptible infected

# this is our agent type
mutable struct SimplePerson
    # state
    belief :: Float64
    know :: Bool
    checker :: Bool
    # other agents this one can infect or be infected by
    contacts :: Vector{SimplePerson}
end

# how we construct a person object
SimplePerson(belief, know,checker) = SimplePerson(belief, know, checker,[])
#SimplePerson() = SimplePerson(susceptible, [])   # default Person is susceptible and has no contacts
#SimplePerson(state) = SimplePerson(state, [])  # default Person has no contacts

SimplePerson

In [10]:
# this is a parametric type
# we can specify which type AGENT is going to be replaced with
# when constructing our Simulation
mutable struct Simulation{AGENT}
    # model parameters:
    # infection rate
    #inf :: Float64
    # recovery rate
    #rec :: Float64
    p :: Float64

    # and this is our population of agents
    pop :: Vector{AGENT}
end

In [11]:
# update an agent
function update!(person, sim)
    if person.know && rand() < sim.p
        other = rand(person.contacts)
        if other.know
            if !other.checker
                if other.belief < 0.5
                    other.belief = (person.belief + other.belief) * 0.5
                end
            end
        else
            other.know = true
            other.belief = person.belief * 0.9 
        end
        
    end
end

update! (generic function with 1 method)

In [12]:
function update_agents!(sim)
    # we need to change the order, otherwise agents at the beginning of the 
    # pop array will behave differently from those further in the back
    order = shuffle(sim.pop)
    
    for p in order
        update!(p, sim)
    end
end   

update_agents! (generic function with 1 method)

In [13]:
# set up a mixed population
# p_contact is the probability that two agents are connected
function setup_mixed(n, p_contact)
    pop = [ SimplePerson(rand(), false,false) for i=1:n ]
    adj = SimpleGraph(n)
    # go through all combinations of agents and 
    # check if they are connected
    for i in eachindex(pop)
        for j in i+1:length(pop)
            if rand() < p_contact
                push!(pop[i].contacts, pop[j])
                push!(pop[j].contacts, pop[i])
                add_edge!(adj, i, j)
                add_edge!(adj, j, i)
            end
        end
    end
    pop,adj
end

setup_mixed (generic function with 1 method)

In [14]:
# TODO add arguments
#adj = SimpleGraph(1000);
#pop,adj = setup_mixed(1000, 0.5) 

#pop[1].know = true
#pop[2].know = true
#pop[3].know = true

# TODO add arguments
#sim = Simulation(1.0, pop)

#for i in 1:20
#    update_agents!(sim)
    
    #println(count(p -> p.know == true, sim.pop)) # prints current number of infected
#end

In [15]:
#                     function parameters after ; have to be 
#                     named on call 
function  setup_sim(;p, N, p_c, n_knowing, n_fact_check, b, seed)
    # for reproducibility
    Random.seed!(seed)

    # create a population of agents, fully mixed
    pop,adj = setup_mixed(N, p_c)

    # create a simulation object with parameter values
    sim = Simulation(p, pop)
    for i in 1:n_knowing
        # one percent of agents are infected
        sim.pop[i].know = true
        sim.pop[i].belief = b
        sim.pop[i].checker = false
    end
    for i in (n_knowing+1):(n_knowing+n_fact_check+1)
        # one percent of agents are infected
        sim.pop[i].know = true
        sim.pop[i].belief = 0
        sim.pop[i].checker = true
    end
    sim,adj
end

setup_sim (generic function with 1 method)

In [18]:
function run_sim(sim, n_steps,n_fact_check, verbose = false)
    # we keep track of the numbers
    knowing = Int[]
    avg_belief = Float64[]
    #remove the checker's impact
    avg_belief_nocheck = Float64[]
    
    n = length(sim.pop)

    # simulation steps
    for t in  1:n_steps
        update_agents!(sim)
        
        push!(knowing, count(p -> p.know == true, sim.pop))
        
        avg = 0
        for i in 1:n
            if sim.pop[i].know
                avg = avg + sim.pop[i].belief
            end
        end
        n_know = count(p -> p.know == true, sim.pop)
        push!(avg_belief, avg /n_know)
        avg = 0
        for i in 1:n
            if sim.pop[i].know && (!sim.pop[i].checker)
                avg = avg + sim.pop[i].belief
            end
        end
        n_know = count(p -> p.know == true, sim.pop) - n_fact_check
        push!(avg_belief_nocheck, avg /n_know)
        
        # a bit of output
        #if verbose
        #    println(t, ", ", n_inf[end], ", ", n_susc[end])
        #end
    end
    
    # return the results (normalized by pop size)
    knowing./n, avg_belief,avg_belief_nocheck
end

run_sim (generic function with 2 methods)

In [ ]:
sim,adj = setup_sim(p=0.1, N=1000, p_c=0.5, n_knowing=10,n_fact_check=10, b=1.0, seed=42)

In [17]:
sim,adj = setup_sim(p=0.1, N=1000, p_c=0.5, n_knowing=10,n_fact_check=10, b=1.0, seed=42)
# inf and susc are arrays containing the proportion of infected/susceptible over time
knowing, avg_belief, avg_belief_nochecker = run_sim(sim, 500,10)

#Plots.plot([knowing, avg_belief], labels = ["knowing (ABM)" "avg belief"])
Plots.plot([knowing, avg_belief_nochecker], labels = ["knowing (ABM)" "avg belief"])

LoadError: type Tuple has no field pop

In [1]:
adj

LoadError: UndefVarError: adj not defined

In [ ]:
closeness_centrality(adj)

In [7]:
tbl = freqtable(avg_belief_nochecker)

LoadError: MethodError: no method matching freqtable(::Array{UnitRange{Int64},1}, ::Int64)
Closest candidates are:
  freqtable(!Matched::AbstractArray{T,1} where T...; skipmissing, weights, subset) at C:\Users\ll1d19\.julia\packages\FreqTables\RBUzx\src\freqtable.jl:132
  freqtable(::Any, !Matched::Union{AbstractString, Symbol}...; args...) at C:\Users\ll1d19\.julia\packages\FreqTables\RBUzx\src\freqtable.jl:194